importing dependencies

In [11]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#show stopwords in english language. we will remove those stopwords. because those words not carry for our project
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

data preprocessing

In [17]:
# loading the dataset to a pandas DataFrame
# Trying different encodings to handle potential encoding issues
try:
    news_dataset = pd.read_csv('/content/train.csv', encoding='utf-8', on_bad_lines='skip') # Try utf-8 first just in case
except UnicodeDecodeError:
    try:
        news_dataset = pd.read_csv('/content/train.csv', encoding='latin1', on_bad_lines='skip') # Try latin1 if utf-8 fails
    except UnicodeDecodeError:
      try:
        news_dataset = pd.read_csv('/content/train.csv', encoding='ISO-8859-1', on_bad_lines='skip') # Try ISO-8859-1 if latin1 fails
      except UnicodeDecodeError:
        news_dataset = pd.read_csv('/content/train.csv', encoding='utf-8', errors='ignore', on_bad_lines='skip') # Skip error lines
        print("Encoding errors encountered, some lines skipped")

In [18]:
news_dataset.shape

(24834, 5)

In [19]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [20]:
#number of missing values
news_dataset.isnull().sum()

,0
id,0
title,676
author,2331
text,47
label,1


In [21]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [23]:
#number of missing values
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [25]:
#margin author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [26]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
24829    Jerome Hudson Rapper T.I.: Trump a âPoster C...
24830    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
24831    Michael J. de la Merced and Rachel Abrams Macy...
24832    Alex Ansary NATO, Russia To Hold Parallel Exer...
24833              David Swanson What Keeps the F-35 Alive
Name: content, Length: 24834, dtype: object


In [27]:
#separating the data and label column
x = news_dataset.drop(columns='label', axis=1)
y = news_dataset['label']

In [28]:
print(x)
print(y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didnât Even...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
24829  20795  ...  Jerome Hudson Rapper T.I.: Trump a âPoster C...
24830  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
24831  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
24832  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
24833  20799  ...            David Swanson What Keeps the F-35 Alive

[24834 rows x 5 columns]
0        1.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
24829    0.0
24830    0.0
24831    0.0
24832    1.0
24833    1.0
Nam

Stemming: process of reducing a word to its root word. remove prefix and suffix.
EX: actor, acting, actress --> act

In [29]:
port_stem = PorterStemmer()

In [33]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)     #all number, upper and lower case single letter convert into " " of content
  stemmed_content = stemmed_content.lower()      #convert all of them into lower case
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]    #removing top words and Stemming process
  stemmed_content = ' '.join(stemmed_content)        #joing the words by space separate A
  return stemmed_content

In [34]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [35]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
24829    jerom hudson rapper trump poster child white s...
24830    benjamin hoffman n f l playoff schedul matchup...
24831    michael j de la merc rachel abram maci said re...
24832    alex ansari nato russia hold parallel exercis ...
24833                            david swanson keep f aliv
Name: content, Length: 24834, dtype: object


In [45]:
x = news_dataset['content'].values
y = news_dataset['label'].values


In [37]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [38]:
print(y)

[1.0 0.0 1.0 ... 0.0 1.0 1.0]


In [46]:
x.shape
y.shape


(24834,)

In [47]:
#converting the textual data into neumerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [61]:
print(x)

  (0, 268)	0.27057472604954497
  (0, 2484)	0.3757521104704707
  (0, 2961)	0.24621403400334252
  (0, 3604)	0.3557230347536848
  (0, 3796)	0.2702151059773696
  (0, 4978)	0.23284760318336842
  (0, 7013)	0.21844512069733138
  (0, 7701)	0.249387812270136
  (0, 8639)	0.2928276833207526
  (0, 8918)	0.35847332612609023
  (0, 13489)	0.2581350440730993
  (0, 15704)	0.28356269323053024
  (1, 1498)	0.29208695621000436
  (1, 1895)	0.15462922113850996
  (1, 2224)	0.38049690981145584
  (1, 2815)	0.18950416530819425
  (1, 3572)	0.2646731667597987
  (1, 5509)	0.7175394627790512
  (1, 6824)	0.18963611790780943
  (1, 16819)	0.298678194506723
  (2, 2945)	0.31919387329279003
  (2, 3105)	0.460244749479103
  (2, 5395)	0.3887168756767809
  (2, 5974)	0.34544540407882596
  (2, 9629)	0.49284792445403086
  :	:
  (24831, 3647)	0.21270554267242012
  (24831, 7051)	0.21691641139042706
  (24831, 8373)	0.2243836812928578
  (24831, 8997)	0.3660449360444374
  (24831, 9527)	0.29565416360714375
  (24831, 9597)	0.1740883731

In [58]:
# Convert y to float, replacing empty strings with np.nan
y = np.array([np.nan if isinstance(val, str) and not val else float(val) for val in y])

# Fill NaN values with 0 and then convert to integer type
y = np.nan_to_num(y, nan=0).astype(int)

In [59]:

print(y)

[1 0 1 ... 0 1 1]


Splitting dataset to traning and test data

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=5)

Training the model Logistic Regration

In [69]:
model = LogisticRegression()

In [70]:
model.fit(x_train, y_train)

LogisticRegression()

Evaluation

In [71]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)


In [72]:
print('Accuracy score of training data:', training_data_accuracy)

Accuracy score of training data: 0.9881713394070569


In [73]:
#accuracy score on the testing data
x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [74]:
print('Accuracy score of training data:', testing_data_accuracy)

Accuracy score of training data: 0.980471109321522


Making a predicting system

prediction 1-->news is fake
prediction 0-->news is real

In [82]:
x_new = x_test[1]

prediction = model.predict(x)
print(prediction)

if(prediction[1]==0):
  print('News is real.')
else:
  print('News is fake')


[1 0 1 ... 0 1 1]
News is real.
